In [1]:
# Clasificación de imágenes uitilizando redes neuronales convolucionales.
# Librerías Necesarias
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import matplotlib.pyplot as plt
import math

In [3]:
# Parámetros de los Datos
PIXEL_SIZE = 255
filas = 2
columnas = 8
MATRIX_SIZE = filas * columnas
IMG_SIZE = 28
BATCH_SIZE = 32
NUM_EPOCHS = 100
TRAIN_SIZE = 1000
TEST_SIZE = 10

In [4]:
# Configuración para mi Data Augmentation utilizando ImageDataGenerator de Keras
rango_rotacion = 30
rango_ancho = 0.25
rango_alto = 0.25
rango_inclinacion = 15
rango_brillo = (50, 1)
rango_acercamiento=[0.5,1.5]
rango_alejamentos=[1.5,0.5]

In [5]:
# Recibir los pixeles y extraer las características y clasificar con capas regulares
# Cargar los datos de MNIST
#(X_train, Y_train), (x_test, y_test) = mnist.load_data()
ruta = "/content/drive/MyDrive/Colab Notebooks/Topicos-en-IA-2021-2/Dataset"
ruta_test = "/content/drive/MyDrive/Colab Notebooks/Topicos-en-IA-2021-2/Test"

data_gen_entrenamiento = ImageDataGenerator(
    rotation_range=rango_rotacion,
    width_shift_range=rango_ancho,
    height_shift_range=rango_alto,
    brightness_range=rango_brillo,
    shear_range=rango_inclinacion,
    zoom_range=rango_acercamiento
)

data_gen_prueba = ImageDataGenerator(
    rescale=1. / 255
)

training_data_gen = data_gen_entrenamiento.flow_from_directory(
    ruta,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical' #[0,1,2,3,4,5,6,7,8,9]
)

validation_data_gen = data_gen_prueba.flow_from_directory(
    ruta_test,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical' #[0,1,2,3,4,5,6,7,8,9]
)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [6]:
# Definición del Modelo
modelo = tf.keras.models.Sequential([
                                     # Capa de Convolución de 32 filtros
                                     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)), 
                                     # Capa de agrupación máxima
                                     tf.keras.layers.MaxPooling2D(2,2),
                                     # Capa de Convolución de 64 filtros porque aumentará el tamaño de filtros
                                     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                     # Capa de agrupación máxima
                                     tf.keras.layers.MaxPooling2D(2,2),
                                     # Capa para hacer dropout para aumentar la velocidad y reducir la cantidad de info que no proporcione información relevante           
                                     tf.keras.layers.Dropout(0.5),
                                     # Capa para vectorizar todo dentro de un vector simple
                                     tf.keras.layers.Flatten(),
                                     # Capa densa oculta para agrupación
                                     tf.keras.layers.Dense(100, activation='relu'),
                                     # Capa densa de salida(0-9) para mostrar el resultado
                                     tf.keras.layers.Dense(10, activation="softmax")
                                     ])
modelo

In [7]:
# Definición del optimizador, función de pérdida y la métrica
modelo.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
# Entrenamiento del Modelo
print("ENTRENANDO EL MODELO...")
training = modelo.fit(
    data_gen_entrenamiento,
    epochs = NUM_EPOCHS,
    batch_size = BATCH_SIZE,
    validation_data=validation_data_gen,
    steps_per_epoch=int(np.ceil(TRAIN_SIZE / float(BATCH_SIZE))), #1000/32=31
    validation_steps=int(np.ceil(TEST_SIZE / float(BATCH_SIZE))) #100/32=3
)
print("MODELO ENTRENADO CON ÉXITO!")

In [ ]:
!pip install tensorflowjs

In [ ]:
!mkdir resultados

In [ ]:
# Exportar el modelo
modelo.save('cnn_model_conv_da_d_100_epoch.h5')
!tensorflowjs_converter --input_format keras cnn_model_conv_da_d_100_epoch.h5 resultados

In [ ]:
!ls -ls 

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import cv2

def prediccion(file):
  #image = load_img(file, target_size=(28,28))
  image =  img_to_array(file)
  image = np.expand_dims(image,axis=0)
  array = modelo.predict(image)
  result = array[0]
  option = np.argmax(result)
  print("Esa imagen es el numero: ", option)
  return option

In [ ]:
fig = plt.figure(figsize=(28, 28))
columns = 10
rows = 1
my_imgs=["0.png","1.png","2.png","3.png","4.png","5.png","6.png","7.png","8.png","9.png"]
for i in range(1, columns*rows +1):
    img = cv2.imread(my_imgs[i-1], cv2.IMREAD_GRAYSCALE)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
rate = 0
for i in range(9):
  print("Imagen: ", i+1)
  ruta = str(i+1) + ".png"
  image = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
  pred = prediccion(image)
  if(i == pred):
    rate = rate + 1
  print(pred)
print("El porcentaje el modelo con las imagenes de prueba es del", rate*10,"%")